<img src="https://whatcar.vn/media/2018/09/car-lot-940x470.jpg"/>

## Прогнозирование стоимости автомобиля по характеристикам

Задача: спрогнозировать стоимость автомобилей, используя данные 



Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. Для этого, перед обработкой данных, предварительно проведем парсинг данных с сайта auto.ru.

# 0. Import

In [ ]:
from sklearn.base import clone
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import f_classif, mutual_info_classif
from pandas import Series
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import sys
import time
from datetime import datetime
import requests as r
import json
from bs4 import BeautifulSoup
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, RandomizedSearchCV
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
import pandas_profiling
import warnings
warnings.simplefilter('ignore')

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 76

## 0.1. Функции, используемые в проекте

In [ ]:
# Средняя абсолютная ошибка в %
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# 1. Парсинг данных с сайта auto.ru

Весь парсинг зацитирован в силу того, что выполнен заранее на локальной машине и результат выгружен в проект. Сам парсинг выполняется очень долго.

In [ ]:
# Начальный URL для дальнейшего парсинга
URL = 'https://auto.ru/moskva/cars/used/'
# Пустой список со ссылками на автомобили
pages_url_list = []

In [ ]:
# Набирает список из ссылок на авто
"""while len(pages_url_list)<5000: 
    for i in range(1,99):   
        response = r.get( URL+'?page=%s'%i)#необходимо для смены страницы
        page = BeautifulSoup(response.text, 'html.parser')
        for link in page.findAll('a',{"class": "Link OfferThumb"}):
           #Проверяем условие присутствия в списке проверяемой ссылки
            try:
                if link['href'] not in pages_url_list: 
                    pages_url_list.append(link['href'])
            except:
                pass
    print(len(pages_url_list))   
    #ждем 1 секунду до следующего запроса
    time.sleep(1)"""

Теперь, когда мы собрали список из ссылок на авто, можно спарсить все данные по этим ссылкам:

In [ ]:
# создаем пустой список с данными по авто
"""cars_list = []
#используем наш список со ссылками
for item in pages_url_list:
   response = r.get(item)
    response.encoding = 'utf8'
    page = BeautifulSoup(response.text, 'html.parser')
   #загружаем страницу
    if page.find('div', class_='CardSold') == None:
       try:
            json_data = json.loads(
                page.find('script', type="application/ld+json").string)
        except:
            print(item)
            pass
        cls_str = 'CardInfoRow_'
        span_str = 'CardInfoRow__cell'
        #Заполняем словарь с данными по каждому автомобилю и добавляем в список с авто
        try:
            cars_list.append({
            'bodyType': json_data['bodyType'],
            'brand': json_data['brand'],
            'car_url': json_data['offers']['url'],
            'color': json_data['color'],
            'description': json_data['description'],
            'engineDisplacement': json_data['vehicleEngine']['engineDisplacement'],
            'enginePower': json_data['vehicleEngine']['enginePower'],
            'fuelType': json_data['fuelType'],
            'image': json_data['image'],
            'mileage': page.find(
                'li', class_=cls_str+'kmAge').find_all('span')[1].text,
            'modelDate': json_data['modelDate'],
            'model_name': json_data['name'],
            'name': json_data['vehicleEngine']['name'],
            'numberOfDoors': json_data['numberOfDoors'],
            'priceCurrency': json_data['offers']['priceCurrency'],
            'productionDate': json_data['productionDate'],
            'vehicleConfiguration': json_data['vehicleConfiguration'],
            'vehicleTransmission': json_data['vehicleTransmission'],
            'Владельцы': page.find(
                'li', class_=cls_str+'ownersCount').find_all('span')[1].text,
            'ПТС': page.find(
                'li', class_=cls_str+'pts').find_all('span')[1].text,
            'Привод': page.find(
                'li', class_=cls_str+'drive').find_all('span')[1].text,
            'Руль': page.find(
                'li', class_=cls_str+'wheel').find_all('span')[1].text,
            'Состояние': page.find(
                'li', class_=cls_str+'state').find_all('span')[1].text,
            'Таможня': page.find(
                'li', class_=cls_str+'customs').find_all('span')[1].text,
            'price': page.find(
                'span', class_='OfferPriceCaption__price').text
            })
        except:
            print(item)
            pass

#Таймер
    print('Ожидаю 0.5 секунды...')
    time.sleep(0.5)"""

Теперь приведем полученные данные к удобному формату и проверим дубликаты

In [ ]:
"""df = pd.DataFrame(cars_list)
#Сохраним в csv формате
df.to_csv('cars_autoru.csv', index=False)"""

# 2. Setup

Загружаем исходные данные для обучения, предложенные условиями задания:

In [ ]:
VERSION = 16
DIR_TRAIN = '../input/parsing1/'  # Дата-Сет из парсинга
DIR_TEST = '../input/sf-dst-car-price-prediction/'
VAL_SIZE = 0.20   # 20%

# 3. Data

In [ ]:
!ls '../input'

Теперь загрузим фреймы:

In [ ]:
train = pd.read_csv(DIR_TRAIN+'cars_auto_ru.csv')
test = pd.read_csv(DIR_TEST+'test.csv')
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

Проверим, все ли получилось:

In [ ]:
train.head(5)

In [ ]:
train.info()

In [ ]:
train.isna().sum()

**Важное замечание:** Парсинг выполнялся на локальной машине, выполнялся очень долго.

Проверим тестовый фрейм:

In [ ]:
test.head(5)

In [ ]:
test.info()

проверим пропуски в тестовом фрейме:

In [ ]:
test.isna().sum()

## 3.1. Data Preprocessing

К сожалению спарсить получилось далеко не все данные, которые находятся в тестовом дата-фрейме. Удалим некоторые из признаков, которые на наш взгляд не несут нагрузки:

In [ ]:
test.drop(['complectation_dict', 'equipment_dict', 'model_info',
           'super_gen', 'vendor', 'Владение'], axis=1, inplace=True)

Теперь посмотрим, что вышло:

In [ ]:
test.info()

Теперь посмотрим, есть ли дубликаты в тренировочном дата-сете:

In [ ]:
train['car_url'].duplicated().sum()

Удалим их:

In [ ]:
# Удалим, указав столбец, по которому смотреть дубликаты
train = train.drop_duplicates(subset=['car_url'])

Проверим, везде ли в тренировочном фрейме есть цена:

In [ ]:
train.price.isna().sum()

Цена присутствует у всех строк во фрейме.

Как мы видим, в некоторых столбцах присутствуют пропуски, с которыми мы в дальнейшем проведем работу и решим, как их заполнить. В целом мы имеем примерно одинаковые по размеру тестовый и тренировочный дата-сеты, примерно по 34 тысячи строк. Теперь, когда первичная обработка данных закончена, можно перейти к более сложным и интересным моментам.

## 3.2. Подготовка и Feature Engeneering

Теперь стоит сформировать списки, в которых будут храниться интересные нам признаки. Их, как обычно, будет три: бинарные, числовые и категориальные:

In [ ]:
# Бинарные признаки - ноль или единица
bin_cols = []

# Категориальные признаки
cat_cols = []

# числовые признаки
num_cols = []

Теперь пойдем слева-напаво по нашим столбцам и рассмотрим, что мы можем выделить из них для дальнейшего обучения модели.

### 3.2.1. Столбец bodyType

Данный столбец отражает тип кузова автомобиля. Посмотрим на количество вариантов:

In [ ]:
train.bodyType.unique()

Видно, что многие названия на самом деле одинаковые, различие только в указании дверей. Попробуем преобразовать названия:

In [ ]:
# Выбираем первое слово для описания типа кузова
train['bodyType'] = train['bodyType'].astype(
    str).apply(lambda x: None if x.strip() == '' else x)
# Понижаем регистр первого слова
train['bodyType'] = train.bodyType.apply(lambda x: x.split(' ')[0].lower())

In [ ]:
train.bodyType.unique()

Вариантов стало меньше, посмотрим на их общее количество:

In [ ]:
train.bodyType.value_counts()

Теперь проделаем то же самое для тестового фрейма.

In [ ]:
# Смотрим уникальные значения
test.bodyType.unique()

In [ ]:
# Выбираем первое слово для описания типа кузова
test['bodyType'] = test['bodyType'].astype(str).apply(
    lambda x: None if x.strip() == '' else x)
# Понижаем регистр первого слова
test['bodyType'] = test.bodyType.apply(lambda x: x.split(' ')[0].lower())

Посмотрим на результаты:

In [ ]:
test.bodyType.unique()

In [ ]:
test.bodyType.value_counts()

Данный признак можно отнести к категориальным, что мы и сделаем:

In [ ]:
cat_cols.append('bodyType')

### 3.2.2. Столбец Brand

Данный столбец отражает название фирмы-производителя автомобиля,посмотрим, нет ли там пропусков:

In [ ]:
train.brand.isna().sum()

Пропусков нет, следовательно столбец можно обрабатывать дальше. Посмотрим на уникальные значения:

In [ ]:
train.brand.unique()

Посмотрим на количество брендов того или иного авто:

In [ ]:
train.brand.value_counts()

На основе предоставленных данных, можно сказать, что на сайте также представлены единичные объявления. Попробуем уменьшить количество брендов, оставив только те, вклад которых составляет не менее 2% дата-сета. Остальные запишем в Other.

In [ ]:
all_brands = len(train.brand)
brands_train = train.brand.value_counts()

Теперь вычислим процентное соотношение каждого бренда во всем списке:

In [ ]:
brand_to_all_train = brands_train/all_brands*100

Посмотрим результат:

In [ ]:
brand_to_all_train

Запишем все, что меньше 2% в Other:

In [ ]:
train['brand_new'] = train.brand.apply(
    lambda x: x if brand_to_all_train[x] > 2 else 'other')

Результат:

In [ ]:
train.brand_new.value_counts()

Так-как наша задача предсказать правильно цену для тестовой выборки, посмотрим, попали ли какие-либо модели из выборки в категорию "other"

In [ ]:
test['brand_new'] = train.brand.apply(
    lambda x: x if brand_to_all_train[x] > 2 else 'other')

In [ ]:
test.brand_new.value_counts()

Как мы можем увидеть, все модели остались в изначальном списке. Данный столбец можно отнести к категориальным, но предварительно посмотрим графически, каких брендов больше:

In [ ]:
train.brand_new.value_counts().plot.barh()

Видно, что бренды, которые мы отнесли к Other теперь находятся на 6 месте по количеству упоминаний. Но без таких манипуляций возникло бы слишком много уникальных значений. Запишем признак в cat_cols:

In [ ]:
cat_cols.append('brand_new')

### 3.2.3. Столбец car_url

Данный признак хранит ссылку на объявление о продаже автомобиля. Для обучения использовать не будем, так как смысловой нагрузки он не несет.

### 3.2.4. Столбец color

Данный признак отражает цвет продаваемого автомобиля. Посмотрим количество уникальных значений:

In [ ]:
train.color.nunique()

Сделаем то же самое для тестовой выборки:

In [ ]:
test.color.nunique()

Проверим на пропуски и добавим к категориальным признакам:

In [ ]:
print(train.color.isna().sum())
print(test.color.isna().sum())

Пропусков нет, добавляем к категориальным признакам.

In [ ]:
cat_cols.append('color')

### 3.2.5. Столбец description

Данный признак содержит описание продовца о продоваемом автомобиле. Проверим есть ли пропуски:

In [ ]:
print(train.description.isna().sum())
print(test.description.isna().sum())

Пропусков нет, создадим признак, который бы отражал количество символов в описании:

In [ ]:
train['comment_length'] = train.description.apply(lambda x: len(str(x)))

повторим для тестового фрейма:

In [ ]:
test['comment_length'] = test.description.apply(lambda x: len(str(x)))

Теперь добавим к числовым признакам:

In [ ]:
num_cols.append('comment_length')

### 3.2.6. Столбец EngineDisplacement

Этот признак описывает объем двигателя авто. Посмотрим количество уникальных значений :

In [ ]:
train.engineDisplacement.unique()

Повторим для тестовой выборки:

In [ ]:
test.engineDisplacement.unique()

Мы видим, что признак отражает литраж двигателя с припиской LTR. Попробуем ее убрать:

In [ ]:
train.engineDisplacement = train.engineDisplacement.apply(lambda x: x[:3])

In [ ]:
train.engineDisplacement.unique()

Видно, что есть значения LT, которые остались неизменными после преобразования. Заменим наиболее частым значением:

In [ ]:
train.engineDisplacement = train.engineDisplacement.replace(
    ' LT', train.engineDisplacement.mode()[0])

Проверим:

In [ ]:
train.engineDisplacement.unique()

Теперь все впорядке. Проверим для тестовой выборки:

In [ ]:
test.engineDisplacement.unique()

Проведем те же преобразования:

In [ ]:
test.engineDisplacement = test.engineDisplacement.apply(lambda x: x[:3])

In [ ]:
#Теперь уберем LT
test.engineDisplacement = test.engineDisplacement.replace(
    ' LT', test.engineDisplacement.mode()[0])

In [ ]:
test.engineDisplacement.unique()

преобразуем признак из обоих фреймов в числовой:

In [ ]:
train.engineDisplacement = train.engineDisplacement.apply(lambda x: float(x))
# Тестовый
test.engineDisplacement = test.engineDisplacement.apply(lambda x: float(x))

Посмотрим на распределение признака:

In [ ]:
train.engineDisplacement.hist()

Для тестовой выборки:

In [ ]:
test.engineDisplacement.hist()

Как мы видим, распределения похожи, основную часть предложений составляют авто с объемом двигателя от 1.3 до 2.5 литра. Никаких выбросов не наблюдается.

Теперь добавим признак к числовым столбцам:

In [ ]:
num_cols.append('engineDisplacement')

### 3.2.7. Столбец enginePower

Признак отражает мощность двигателя автомобиля. Посмотрим на уникальные значения:

In [ ]:
train.enginePower.unique()

Попробуем избавиться от дополнительной информации в обозначении:

In [ ]:
train['enginePower'] = train['enginePower'].apply(
    lambda x: x[:1] if len(x) == 5 else (x[:2] if len(x) == 6 else x[:3]))

Проверим результат:

In [ ]:
train.enginePower.unique()

Проделаем то же самое для тестового фрейма:

In [ ]:
test.enginePower.unique()

Теперь уберем ненужную информацию:

In [ ]:
test['enginePower'] = test['enginePower'].apply(
    lambda x: x[:1] if len(x) == 5 else (x[:2] if len(x) == 6 else x[:3]))

Результат:

In [ ]:
test.enginePower.unique()

Теперь преобразуем в числовой формат:

In [ ]:
train['enginePower'] = train['enginePower'].apply(lambda x: int(x))
test['enginePower'] = test['enginePower'].apply(lambda x: int(x))

Посмотрим распределение признака:

In [ ]:
#Тренировочная выборка
train.enginePower.hist()

In [ ]:
#Тестовая выборка
test.enginePower.hist()

Видно что большую часть представляют двигатели с мощностью от 100 до 250 л.с. Графики немного сдвинуты влево, явных выбросов не наблюдается.

Добавим признак в числовые:

In [ ]:
num_cols.append('enginePower')

### 3.2.8. Столбец fuelType

Данный признак содержит данные о типе топлива, используемого в автомобиле. Посмотрим на уникальные значения:

In [ ]:
print(train.fuelType.unique())
print(test.fuelType.unique())

Теперь посмотрим, есть ли в данных пропуски:

In [ ]:
print(train.fuelType.isna().sum())
print(test.fuelType.isna().sum())

Пропусков нет,что очень радует. Теперь посмотрим распределение признака для обеих выборок:

In [ ]:
#Тренировочная выборка
train.fuelType.hist()

In [ ]:
#Тестовая выборка
test.fuelType.hist()

Графики выборок схожи, основное количество представленных автомобилей работает на бензиновом топливе. Так как предварительной обработки признаку не требуется, добавим его в категориальные:

In [ ]:
cat_cols.append('fuelType')

### 3.2.9. Столбец image

Данный признак содержит фото автомобиля. На мой взгляд особой информативности не несет, поэтому для обучения его брать не будем.

### 3.2.10. Столбец mileage

Признак содержит данные о пробеге продаваемого авто. Проверим есть ли пропуски:

In [ ]:
print(train.mileage.isna().sum())
print(test.mileage.isna().sum())

Пропусков нет, проверим сами данные:

In [ ]:
train.mileage.unique()

Видно, что данные грязные. Уберем из данных лишний мусор:

In [ ]:
# убираем \xa0 из данных
train['mileage'] = train['mileage'].apply(lambda x: str(x).replace('\xa0', ''))
# убираем км из данных
train['mileage'] = train['mileage'].apply(lambda x: str(x).replace('км', ''))

Проверим результат:

In [ ]:
train.mileage.unique()

Замечательно! Мы убрали весь мусор, который мешал работать с данными. Проделаем то же самое для тестовой выборки:

In [ ]:
# убираем \xa0 из данных
test['mileage'] = test['mileage'].apply(lambda x: str(x).replace('\xa0', ''))
# убираем км из данных
test['mileage'] = test['mileage'].apply(lambda x: str(x).replace('км', ''))

Теперь преобразуем в числовой формат:

In [ ]:
# Тестовая выборка
test['mileage'] = test['mileage'].apply(lambda x: int(x))
# Обучающая выборка
train['mileage'] = train['mileage'].apply(lambda x: int(x))

Посмотрим распределение значений признака для обеих выборок:

In [ ]:
# Тренировочная выборка
train.mileage.hist()

In [ ]:
# Тестовая выборка
test.mileage.hist()

Теперь посмотрим на основные показатели признака:

In [ ]:
train.mileage.describe()

In [ ]:
test.mileage.describe()

Распределение на графике может считаться  нормальным, максимальное значение равно 1 млн. км. Является ли это выбросом? Вполне возможно, но есть автомобили, у которых пробег может быть еще больше. Такой вывод делаю, основываясь на собственном опыте. Для достоверности - можно посмотреть, какое количество значений привышается исходя из программного метода:

In [ ]:
IQR = train['mileage'].quantile(0.75) - train['mileage'].quantile(0.25)
perc25 = train['mileage'].quantile(0.25)  # 25-й перцентиль
perc75 = train['mileage'].quantile(0.75)  # 75-й перцентиль

print(
    '25-й перцентиль: {},'.format(perc25),
    '75-й перцентиль: {},'.format(perc75),
    "IQR: {}, ".format(IQR),
    "Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR,
                                           l=perc75 + 1.5*IQR))

Как мы видим, исходя из программного метода верхней границей является 371107 км. Но здравый смысл подсказывает, что могут быть значения и выше, и они вполне реальны. Посмотрим сколько значений привышает границу:

In [ ]:
train.mileage[train.mileage > 371107].count()

Как мы видим значений очень мало, что логично. Оставим их без изменений. Посмотрим аналогично для тестовой выборки:

In [ ]:
IQR = test['mileage'].quantile(0.75) - test['mileage'].quantile(0.25)
perc25 = test['mileage'].quantile(0.25)  # 25-й перцентиль
perc75 = test['mileage'].quantile(0.75)  # 75-й перцентиль

print(
    '25-й перцентиль: {},'.format(perc25),
    '75-й перцентиль: {},'.format(perc75),
    "IQR: {}, ".format(IQR),
    "Границы выбросов: [{f}, {l}].".format(f=perc25 - 1.5*IQR,
                                           l=perc75 + 1.5*IQR))

Для теста верхний порог оказался даже выше, 400769 км. Посмотрим сколько значений его привышают:

In [ ]:
test.mileage[test.mileage > 400769].count()

Значений также мало, оставим их без изменений. Добавим признак к числовым:

In [ ]:
num_cols.append('mileage')

### 3.2.11. Столбец modelDate

Данный признак содержит информацию о годе производства модели авто. Посмотрим пропуски:

In [ ]:
print(train.modelDate.isna().sum())
print(test.modelDate.isna().sum())

Перейдем к просмотру данных:

In [ ]:
train.modelDate.unique()

In [ ]:
test.modelDate.unique()

Посмотрим график распределения признака:

In [ ]:
# тренировочная выборка
train.modelDate.hist()

In [ ]:
# тестовая выборка
test.modelDate.hist()

Как видно из графика, большинство авто, представленые в дата-сете, возрастом 20 и менее лет. Хотя есть и раритеты. Выделим на основе имеющихся данных новый признак, который отражает время, прошедшее с момента производства модели (Текущий год 2021):

In [ ]:
train['model_time'] = datetime.now().year - train.modelDate

Проверим, что получилось:

In [ ]:
train.model_time.unique()

Повторим для тестовой выборки:

In [ ]:
test['model_time'] = datetime.now().year - test.modelDate

Проверим результат:

In [ ]:
test.model_time.unique()

Добавим новый признак к числовым:

In [ ]:
num_cols.append('model_time')

### 3.2.12 Столбец model_name

Данный признак отражает название модели авто. Посмотрим на пропуски:

In [ ]:
print(train.model_name.isna().sum())
print(test.model_name.isna().sum())

Пропусков нет, посмотрим на сами данные:

In [ ]:
train.model_name.value_counts()

Попробуем привести значения к нижнему регистру и удалить лишние пробелы:

In [ ]:
train['model_name'] = train['model_name'].apply(
    lambda x: x.lower().strip())
train['model_name'].value_counts()

Как мы видим,решение почти не помогло. Но, тем не менее, применим его и к тестовой выборке:

In [ ]:
# Посчитаем количество упоминаний каждого значения в тестовой выборке
test.model_name.value_counts()

In [ ]:
test['model_name'] = test['model_name'].apply(
    lambda x: x.lower().strip())
test['model_name'].value_counts()

И здесь особого результата не дало. Запишем признак в категориальные:

In [ ]:
cat_cols.append('model_name')

### 3.2.13. Столбец name

Признак содержит расширенные данные. Посмотрим данные:

In [ ]:
train.name.value_counts()

In [ ]:
test.name.value_counts()

Видно, что в данных содержится непонятная "каша" из других столбцов. Таким образом данный столбец только дублирует данные. Смысловой нагрузки не несет и для обучения использоваться не будет.

### 3.2.14. Столбец numberOfDoors

Признак содержит информацию о количестве дверей авто. Проверим на пропуски:

In [ ]:
print(train.numberOfDoors.isna().sum())
print(test.numberOfDoors.isna().sum())

Посмотрим сами данные:

In [ ]:
# Обучающая выборка
train.numberOfDoors.value_counts()

In [ ]:
# Тестовая выборка
test.numberOfDoors.value_counts()

А вот в тестовой выборке возникло совершенно внезапно значение 0. Либо это бездверная капсула, либо опечатка. Я надеюсь опечатка. Заменим значение медианой:

In [ ]:
test['numberOfDoors'] = test['numberOfDoors'].apply(
    lambda x: int(test['numberOfDoors'].median()) if x == 0 else int(x))

Проверим результат:

In [ ]:
test.numberOfDoors.value_counts()

Посмотрим графики распределения значений признака:

In [ ]:
# Тренировочная выборка
train.numberOfDoors.hist()

In [ ]:
# Тестовая выборка
test.numberOfDoors.hist()

Видно, что распределения схожи. Наибольшее количество объявлений о продаже авто содержат пятидверные автомобили. Теперь добавим признак к категориальным:

In [ ]:
cat_cols.append('numberOfDoors')

### 3.2.15. Столбец parsing_unixtime

Данный признак содержит информацию о времени, когда была спарсена информация об авто. Присутствует только в тестовой выборке. Поэтому использовать для обучения не будем.

### 3.2.16. Столбец priceCurrency

Признак содержит информацию о том, какая валюта используется при продаже. Посмотрим на данные:

In [ ]:
print(train.priceCurrency.unique())
print(test.priceCurrency.unique())

Как мы видим признак принимает всего одно значение "RUB". Особой информативности признак не несет и в обучении роли не сыграет, поэтому пропустим его.

### 3.2.17. Столбец productionDate

Признак содержит информацию о годе производства авто. Проверим пропуски:

In [ ]:
print(train.productionDate.isna().sum())
print(test.productionDate.isna().sum())

Пропусков нет, проверим сами данные:

In [ ]:
train.productionDate.unique()

И для тестовой выборки:

In [ ]:
test.productionDate.unique()

На всякий случай проверим корреляцию между productionDate и modelDate

In [ ]:
train[['modelDate', 'productionDate']].corr()

In [ ]:
test[['modelDate', 'productionDate']].corr()

Корреляция очень высокая, значит один признак почти дублирует другой, а это может негативно сказаться на обучении. Во избежание этого, оставим только один из признаков, то есть modelDate, который мы добавили выше.

### 3.2.18. Столбец sell_id

Признак содержит уникальный идентификатор объявления о продаже. Особой информативности не несет, поэтому в дальнейшем использоваться не будет. В дополнение - присутствует только в тестовой выборке.

### 3.2.19. Столбец vehicleConfiguration

Признак содержит информацию о конфигурации транспортного средства. Посмотрим данные:

In [ ]:
train.vehicleConfiguration.value_counts()

In [ ]:
test.vehicleConfiguration.value_counts()

Как мы видим столбец дублирует данные из других признаков. В этом случае - информативности он не несет, поэтому при обучении использоваться не будет.

### 3.2.20. Столбец vehicleTransmission

Признак содержит информацию о типе коробки передач, используемой в транспортном средстве. Проверим на пропуски:

In [ ]:
print(train.vehicleTransmission.isna().sum())
print(test.vehicleTransmission.isna().sum())

Пропусков не наблюдается, посмотрим уникальные значения для обеих выборок:

In [ ]:
print(train.vehicleTransmission.unique())
print(test.vehicleTransmission.unique())

Признак принимает всего 4 значения, пропусков нет, может быть добавлен к категориальным признакам. Посмотрим на распределение признака:

In [ ]:
# Тренировочная выборка
train.vehicleTransmission.hist()

In [ ]:
# Тестовая выборка
test.vehicleTransmission.hist()

Из графика видно, что чаще всего встречается автоматическая коробка передач. Добавляем признак к категориальным:

In [ ]:
cat_cols.append('vehicleTransmission')

### 3.2.21. Столбец Владельцы

Признак содержит информацию о количестве владельцев авто. Посмотрим пропуски:

In [ ]:
print(train.Владельцы.isna().sum())
print(test.Владельцы.isna().sum())

Пропусков нет, посмотрим на уникальные значения:

In [ ]:
# Обучающая выборка
train.Владельцы.unique()

In [ ]:
# Тестовая выборка
test.Владельцы.unique()

Как мы видим, данные загрязнены. Попробуем очистить их:

In [ ]:
train['Владельцы'] = train['Владельцы'].apply(
    lambda x: str(x).replace('\xa0', ' '))

Проверим, что получилось:

In [ ]:
train.Владельцы.unique()

Все работает как и должно. Применим к тестовой выборке аналогичный прием:

In [ ]:
test['Владельцы'] = test['Владельцы'].apply(
    lambda x: str(x).replace('\xa0', ' '))

Убедимся, что все хорошо:

In [ ]:
test.Владельцы.unique()

Посмотрим распределение признака для обеих выборок:

In [ ]:
# Обучающая выборка
train.Владельцы.hist()

In [ ]:
# Тестовая выборка
test.Владельцы.hist()

Как мы видим, преобладают авто, у которых было 3 и более владельцов, но в целом - различие с другими значениями не настолько сильное в случае тренировочной выборки. Добавим признак к категориальным:

In [ ]:
cat_cols.append('Владельцы')

### 3.2.22. Столбец ПТС

Данный признак содержит информацию о ПТС автомобиля. Проверим на пропуски:

In [ ]:
print(train.ПТС.isna().sum())
print(test.ПТС.isna().sum())

У нас есть один пропуск в тестовой выборке. Посмотрим на значения, принимаемые признаком, чтобы понять, что мы можем сделать:

In [ ]:
train.ПТС.unique()

Как мы видим, признак принимает два значения. Посмотрим тестовую выборку:

In [ ]:
test.ПТС.unique()

Заменим пропуск значением моды для данного признака:

In [ ]:
test['ПТС'] = test['ПТС'].apply(
    lambda x: test['ПТС'].mode()[0] if pd.isna(x) else x)

Просмотрим результат:

In [ ]:
test.ПТС.unique()

Посмотрим распределение признака:

In [ ]:
# Тренировочная выборка
sns.countplot(x='ПТС', data=train)

In [ ]:
# Тестовая выборка
sns.countplot(x='ПТС', data=test)

Из распределения видно, что оригиналов ПТС гораздо больше чем дубликатов в обеих выборках. Данный признак является бинарным. Добавим его:

In [ ]:
bin_cols.append('ПТС')

### 3.2.23. Столбец Привод

Признак содержит информацию о том, какой привод у транспортного средства. Проверим на пропуски:

In [ ]:
print(train.Привод.isna().sum())
print(test.Привод.isna().sum())

Пропусков нет. Просмотрим сами данные:

In [ ]:
train.Привод.unique()

Повторим для тестовой выборки:

In [ ]:
test.Привод.unique()

Признак категориальный, посмотрим распределение значений:

In [ ]:
# Тренировочная выборка
sns.countplot(x='Привод', data=train)

In [ ]:
# Тестовая выборка
sns.countplot(x='Привод', data=test)

Распределение признаков для обеих выборок похож. Наибольший процент в дата-сетах составляют передний и полный привод. Дальнейшей обработки не требуется, добавим признак к остальным категориальным:

In [ ]:
cat_cols.append('Привод')

### 3.2.24. Столбец Руль

Признак отражает положение руля в автомобиле. Просмотрим данные:

In [ ]:
# Тренировочная выборка
train.Руль.unique()

In [ ]:
# Тестовая выборка
test.Руль.unique()

Пропусков нет, признак является бинарным. Посмотрим его распределение для обеих выборок:

In [ ]:
# Тренировочная выборка
sns.countplot(x='Руль', data=train)

In [ ]:
# Тестовая выборка
sns.countplot(x='Руль', data=test)

Из графика видно, что преобладают автомобили с левым рулем. Дальнейшей обработки признак не требует, добавим к бинарным:

In [ ]:
bin_cols.append('Руль')

### 3.2.25. Столбец Состояние

Признак описывает состояние авто. Просмотрим данные:

In [ ]:
#Тренировочная выборка
train.Состояние.unique()

In [ ]:
#Тестовая выборка
test.Состояние.unique()

В тестовой выборке только одно значение признака, однако в тренировочной два. Посмотрим распределение для тренировочной выборки:

In [ ]:
# Тренировочная выборка
sns.countplot(x='Состояние', data=train)

Посчитаем значения:

In [ ]:
train.Состояние.value_counts()

Как видно и из распределения и проверки количества значений, признак все же является бинарным, хотя второе значение присутствует в ничтожно малом количестве. Добавим к бинарным:

In [ ]:
bin_cols.append('Состояние')

### 3.2.26. Столбец Таможня

Признак отражает, что автомобиль растоможен, и одинаков для всех строк. Смысловой нагрузки не несет и смысла добавлять его в обучение нет, пропускаем.

### 3.2.27. Целевой столбец price

Искомый признак, отражающий стоимость автомобиля. Присутствует только в тренировочной выборке. Просмотрим уникальные значения и пропуски:

In [ ]:
train.price.isna().sum()

Пропусков нет.

In [ ]:
train.price.unique()

Данные грязные, требуют предварительной очистки. Приступим:

In [ ]:
train['price'] = train['price'].apply(lambda x: str(x).replace('\xa0', ''))

проверим результат:

In [ ]:
train.price.unique()

Пол дела сделано, теперь нужно убрать обозначение валюты:

In [ ]:
train['price'] = train['price'].apply(lambda x: str(x).replace('₽', ''))

Еще раз проверим:

In [ ]:
train.price.unique()

Весь мусор удален, теперь приведем столбец к числовому формату:

In [ ]:
train['price'] = train['price'].apply(lambda x: int(x))

И снова проверим:

In [ ]:
train.price.unique()

Все нормально преобразовалось. Посмотрим распределение целевого признака:

In [ ]:
plt.figure()
plt.title(f"Распределение {'price'}")
sns.distplot(train.price, kde=False)

Как мы видим, большинство цен лежит в диапазоне от 0 до 1 млн. рублей. Помимо этого присутствуют выбросы, но они не противоречат здравому смыслу. Теперь рассмотрим влияние некоторых признаков на цену:

In [ ]:
plt.figure(figsize=(10, 15))
plt.scatter((train.price), train.brand_new)

Что логично, чем "элитнее" авто, тем больше у него становится разброс цен. Также некоторый вклад вносят и авто, которые мы отнесли к типу other. Возможно там имеются раритеты, которые могут много стоить.

Теперь посмотрим зависимость цены от привода:

In [ ]:
plt.figure(figsize=(10, 15))
plt.scatter((train.price), train.Привод)

Наименьший разброс цен наблюдается у переднего привода, а наименее сгруппированным является полный привод. Вероятно это связано с разнообразием различных автомобилей из разных классов, имеющих данную характеристику.

Проверим аналогичным способом, как влияет количество владельцев на стоимость автомобиля:

In [ ]:
plt.figure(figsize=(10, 15))
plt.scatter((train.price), train.Владельцы)

Из графика видно, что с увеличением числа владельцев увеличивается сгруппированность у "меньшей" стоимости автомобиля.

## 3.3. Label Encoding

Для дальнейшей обработки соединим оба фрейма в один:

In [ ]:
train['sample_'] = 0
test['sample_'] = 1
df = pd.concat([test, train])
df = df.reset_index().drop(['index'], axis=1)

Теперь оставим только выделенные признаки из нашего дата-сета, которые будут нужны для обучения:

In [ ]:
df = df[cat_cols + bin_cols + num_cols + ['price', 'sample_']]

Проверим результаты:

In [ ]:
df.head(1)

Теперь приступим непосредственно к самой обработке столбцов

### 3.3.1. Обработка бинарных признаков

Для начала необходимо преобразовать полученные признаки при помощи Label Encoding'а:

In [ ]:
label_encoder = LabelEncoder()
for col in bin_cols:
    df[col] = label_encoder.fit_transform(df[col])

Проверим правильность выполнения:

In [ ]:
df[bin_cols].head(5)

Визуально, преобразовалось все хорошо, но проверим дополнительно при помощи value_counts():

In [ ]:
for col in bin_cols:
    display(df[col].value_counts())

Посмотрим значимость признаков:

In [ ]:
imp_cat = Series(mutual_info_classif(df[df.sample_ == 0][bin_cols], df[df.sample_ == 0]['price'],
                                     discrete_features=True), index=bin_cols)
imp_cat.sort_values(inplace=True)
imp_cat.plot(kind='barh')

Наибольшее влияние на цену оказывает признак ПТС, Состояние же вклада почти не делает. Данная логика является странной, ведь большее влияние должно оказывать как раз Состояние транспортного средства. Скорее всего это связано с тем, что у нас фактически отсутствует второе значение данного признака.

### 3.3.2. Категориальные признаки

На всякий случай еще раз посмотрим количество уникальных значений для каждого признака из списка:

In [ ]:
for col in cat_cols:
    print(f'{col}', df[col].nunique())

Видно что уникальных значений для некоторых переменных довольно много:

Теперь перекодируем признаки при помощи cat.codes на основе имеющихся категориальных столбцов:

In [ ]:
for column in cat_cols:
    df[column] = df[column].astype('category').cat.codes

проверим результат:

In [ ]:
df.head(5)

Все преобразовалось правильно.

Посмотрим значимость категориальных признаков:

In [ ]:
imp_num = Series(f_classif(df[cat_cols][df['sample_'] == 0], df[df['sample_'] == 0]['price'])[0],
                 index=cat_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')

Как мы можем заметить, наибольший вклад осуществляют признаки "bodyType,Владельцы,fuelType". В целом, вклад почти всех признаков является большим.

### 3.3.3. Числовые признаки

Посмотрим на сами числовые признаки и их корреляцию:

In [ ]:
df[df['sample_'] == 0][num_cols + ['price']]

Теперь построим корреляцию:

In [ ]:
plt.figure(figsize=(12, 7))
sns.heatmap(df[df['sample_'] == 0][num_cols + ['price']].corr(), annot=True)

Мы видим, что наибольшая зависимость между признаками enginePower и engineDisplacement. Так как мощность двигателя является более показательным признаком, оставим его, а признак engineDisplacement удалим из выборки и из num_cols.

In [ ]:
num_cols.remove('engineDisplacement')
df = df.drop(['engineDisplacement'], axis=1)

Все остальные признаки находятся в пределах относительной нормы, очень высокой корреляции не наблюдается. Посмотрим на значимость числовых признаков:

In [ ]:
imp_num = Series(f_classif(df[num_cols][df['sample_'] == 0], df[df['sample_'] == 0]['price'])[0],
                 index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')

Влияние всех признаков высокое. Наиболее важными признаками являются model_time и enginePower

Теперь, когда преобразование всех признаков закончено, можно перейти к созданию и обучению моделей.

# 4. Обучение

## 4.1. Train Split

Разделим наш дата-фрейм обратно на тренировочный и тестовый:

In [ ]:
df_train = df[df.sample_ == 0]
df_test = df[df.sample_ == 1].drop(['price', 'sample_'], axis=1).values

Теперь разделим наш тренировочный дата-сет на обучающую и валидационную выборки:

In [ ]:
X = df_train.drop(['price', 'sample_'], axis=1)
Y = df_train['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=VAL_SIZE, random_state=RANDOM_SEED)

## 4.2. Model 1: Создадим "наивную" модель 
Эта модель будет предсказывать среднюю цену по мощности двигателя (enginePower). 
C ней будем сравнивать другие модели.




In [ ]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [ ]:
# Находим median по экземплярам enginepower в трейне и размечаем тест
predict = X_test['enginePower'].map(
    tmp_train.groupby('enginePower')['price'].median())

# оцениваем точность
print(
    f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.2f}%")

Получили ужасную метрику. Посмотрим, что получится дальше.

## 4.3. Model 2 : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

### 4.3.1 Fit

Разделим еще раз на обучающую и валидационную выборки:

In [ ]:
X = df_train.drop(['price', 'sample_'], axis=1).values
Y = df_train['price'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=VAL_SIZE, random_state=RANDOM_SEED)

In [ ]:
model = CatBoostRegressor(iterations=5000,
                          random_seed=RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                          )
model.fit(X_train, y_train,
          eval_set=(X_test, y_test),
          verbose_eval=0,
          use_best_model=True,
          )

model.save_model('catboost_single_model_baseline.model')

In [ ]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Уже лучше, чем у наивной модели. Попробуем логарифмировать.

### 4.3.2. Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log и np.exp).    


In [ ]:
np.log(y_train)

In [ ]:
model = CatBoostRegressor(iterations=5000,
                          random_seed=RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                          )
model.fit(X_train, np.log(y_train+1),
          eval_set=(X_test, np.log(y_test)),
          verbose_eval=0,
          use_best_model=True,
          )

model.save_model('catboost_single_model_2_baseline.model')

In [ ]:
predict_test = np.exp(model.predict(X_test))
print(
    f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность выросла еще, однако недостаточно. Исследуем другие модели.

## 4.4. Другие модели

### 4.4.1. Линейная регрессия

Начнем с самого простого, линейной регрессии:

In [ ]:
linreg = LinearRegression().fit(X_train, np.log(y_train+1))
print(
    f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(linreg.predict(X_test))))*100:0.2f}%")
predict_test = np.exp(linreg.predict(X_test))

Точность все еще плохая, возможно дальше будет лучше.

### 4.4.2. Градиентный бустинг

In [ ]:
grb = GradientBoostingRegressor(min_samples_split=2, learning_rate=0.03,
                                max_depth=10, n_estimators=300, random_state=RANDOM_SEED)
grb.fit(X_train, np.log(y_train+1))
print(
    f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(grb.predict(X_test))))*100:0.2f}%")
predict_test = np.exp(grb.predict(X_test))

Получили значение, немного большее, чем при catBoost

### 4.4.3. RandomForest

Опробуем RandomForest с подбором параметров:

In [ ]:
"""
random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
               'max_features': ['auto', 'sqrt'],
               'max_depth': [int(x) for x in np.linspace(10, 110, num = 11)],
               'min_samples_split': [2, 5, 10],
               'min_samples_leaf': [1, 2, 4],
               'bootstrap': [True, False]}

rf = RandomForestRegressor(random_state=RANDOM_SEED)
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, 
                               cv=3, verbose=2, random_state=RANDOM_SEED, n_jobs=-1)
rf_random.fit(X_train, np.log(y_train+1))
print(f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(rf_random.predict(X_test))))*100:0.2f}%")
predict_test = np.exp(rf_random.predict(X_test))
#Точность модели по метрике MAPE: 15.88%
"""

Получили результат метрики 15.88%. Выполняется ужасно долго, около четырех часов. Пока лучший результат показал градиентный бустинг. Попробуем собрать ансамбль.

### 4.4.4. Ансамбль алгоритмов

In [ ]:
estimators = [('cb', CatBoostRegressor(iterations=5000,
                                       random_seed=RANDOM_SEED,
                                       eval_metric='MAPE',
                                       custom_metric=['R2', 'MAE'],
                                       silent=True)),
              ('rf', RandomForestRegressor(n_estimators=500,
                                           n_jobs=-1,
                                           max_depth=15,
                                           max_features='log2',
                                           random_state=RANDOM_SEED,
                                           oob_score=True)),
              ('lr', LinearRegression(normalize=True,
                                      n_jobs=-1))]


st_ensemble = StackingRegressor(estimators=estimators,
                                final_estimator=GradientBoostingRegressor(
                                    min_samples_split=2,
                                    learning_rate=0.03,
                                    max_depth=10,
                                    n_estimators=500))


st_ensemble.fit(X_train, np.log(y_train+1))


print(
    f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(st_ensemble.predict(X_test))))*100:0.2f}%")
predict_test = np.exp(st_ensemble.predict(X_test))

Получили метрику 14,9%. Возможно не очень хорошая метрика связана с выбором параметров либо их качеством обработки. Я не могу дать точного ответа на этот вопрос, как бы ни хотел. Можно было бы провести еще множество исследований, но не остается времени. Последнее, что попробуем - использовать StandardScaler на данных.

### 4.4.5. Ансамбль с standardScaler

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = y_train
y_test = y_test

Теперь посмотрим, как поведет себя Ансамбль:

In [ ]:
estimators = [('cb', CatBoostRegressor(iterations=5000,
                                       random_seed=RANDOM_SEED,
                                       eval_metric='MAPE',
                                       custom_metric=['R2', 'MAE'],
                                       silent=True)),
              ('rf', RandomForestRegressor(n_estimators=500,
                                           n_jobs=-1,
                                           max_depth=15,
                                           max_features='log2',
                                           random_state=RANDOM_SEED,
                                           oob_score=True)),
              ('lr', LinearRegression(normalize=True,
                                      n_jobs=-1))]


st_ensemble = StackingRegressor(estimators=estimators,
                                final_estimator=GradientBoostingRegressor(
                                    min_samples_split=2,
                                    learning_rate=0.03,
                                    max_depth=10,
                                    n_estimators=500))


st_ensemble.fit(X_train, np.log(y_train+1))


print(
    f"Точность модели по метрике MAPE: {(mape(y_test, np.exp(st_ensemble.predict(X_test))))*100:0.2f}%")
predict_test = np.exp(st_ensemble.predict(X_test))

Как мы видим, почти ничего не поменялось. Лучшие показатели дал только алгоритм Градиентного Бустинга, но разница не является очень сильной. Я склоняюсь к тому что стоит использовать последний алгоритм для решения задачи, поэтому выберем его в качестве решения и приступим к сабмиту

# Submission

Обучим алгоритм на всей выборке:

In [ ]:
st_ensemble.fit(X, np.log(Y+1))

Теперь применим к тестовой выборке:

In [ ]:
# округляем, чтобы были целые числа
predict_submission = np.round(
    np.exp(st_ensemble.predict(df_test)), -3).astype('int')

Готовим и сохраняем сабмит:

In [ ]:
VERSION = 3

In [ ]:
sample_submission['price'] = predict_submission
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

В итоге получили **MAPE 22%**!

Вероятно, признаки, которые мы подобрали не дают достоверного качества при обучении. Либо данных оказалось слишком мало, чтобы нормально обучить. Вариантов может быть очень много, но результат заключается в том, что нам удалось немного улучшить метрику ( Я честно постарался бы сделать это еще лучше, но не осталось времени.). В дальнейшем - стоит попробовать использовать иные преобразования категориальных признаков, возможно выделить большее их количество. А так, как результат мы получили разницу в метрике на тренировочной выборке и тестовой в ~7%. Как напутствие самому себе хочу отметить, что стоит выделять большее количество различных признаков. Возможно они смогли бы сыграть важную роль.

# What's next?
Или что еще можно сделать, чтоб улучшить результат:

* Спарсить свежие данные 
* Посмотреть, что можно извлечь из признаков или как еще можно обработать признаки
* Сгенерировать новые признаки
* Попробовать подобрать параметры модели
* Попробовать другие алгоритмы и библиотеки ML
* Сделать Ансамбль моделей, Blending, Stacking